# for this emailbuilder we will use edenAI

In [29]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.llms import EdenAI
import requests
import json
from types import SimpleNamespace
from sqlalchemy import create_engine, text

_ = load_dotenv(find_dotenv())
edenai_api_key = os.getenv("EDENAI_API_KEY")
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()

In [30]:
def get_answer(prompt):
    url = "https://api.edenai.run/v2/text/chat"

    payload = {
        "response_as_dict": True,
        "attributes_as_list": False,
        "show_original_response": False,
        "temperature": 0.0,
        "max_tokens": 400,
        "providers": "openai",
        "text": f"{prompt}",
    }
    headers = {
        "accept": "application/json",
        "content_type": "application/json",
        "authorization": f"Bearer {edenai_api_key}"
    }

    response = requests.post(url, json=payload, headers=headers)
    responseObject = json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))
    return responseObject.openai.generated_text

In [38]:
# TOEVOEGING

def get_email_prompts(email_list) -> list:

    email_string = ','.join(email_list)

    url = f'https://dep2-g1.vichogent.be/contact?ids={email_string}'

    response = requests.get(url)

    query_list = ["\'" + i + "\'" for i in email_list]
    query_string = ','.join(query_list)

    query = text(f"select ContactpersoonId,FunctieTitel from Contactfiche where ContactpersoonId in ({query_string})")
    print(query)
    result = conn.execute(query).fetchall()
    print(result)
    function_dict = {}
    for row in result:
        function_dict[row[0]] = row[1]


    email_prompts = []

    if response.status_code == 200:
        json_data = response.json()

        for user_data in json_data:
            user = user_data['cantact']
            
            function = function_dict[user]
            
            campaigns = user_data['CampagneNaam']

            if campaigns:
                campaigns_list = ', '.join(campaigns.values())
                prompt = f"Create an email written for {user} that executes the following function: {function} and recommend the following campaigns: {campaigns_list}."
            else:
                prompt = f"Create an email written for {user} that executes the following function: {function} but there are no campaigns to recommend."

            email_prompts.append(prompt)

    else:
        error = f"Error: {response.status_code} - {response.text}"
        return list(error)
    return email_prompts
        
# EINDE TOEVOEGING

In [39]:
prompts = get_email_prompts(['C39094D7-60B1-EB11-811F-001DD8B72B62', '9D75B0F8-72D7-EB11-8121-001DD8B72B61'])

print(prompts)
user= "C39094D7-60B1-EB11-811F-001DD8B72B62"
function= 'Verantwoordelijke Personeel'
campaigns = ['D9834556-240C-EE11-8F6E-6045BD8956C9', 'B3865100-DEEF-ED11-8849-6045BD895233']

prompt= f"Create an email written for Erik Van Vliet that executes the following function: {function} and recommend the following campaigns: {campaigns}. You do not want an answert as this is an automated mail"

# text= f"Create an email written for {user} that executes the following function: {function} and recommend the following campaigns: {campaigns}"

# text= f"schrijf een email naar {user} die binnen het bedrijf de volgende functie heeft: {function} en biedt volgende campagnes aan: {campaigns}"

select ContactpersoonId,FunctieTitel from Contactfiche where ContactpersoonId in ('C39094D7-60B1-EB11-811F-001DD8B72B62','9D75B0F8-72D7-EB11-8121-001DD8B72B61')
[('9D75B0F8-72D7-EB11-8121-001DD8B72B61', 'Operations & Enzblement Lead'), ('C39094D7-60B1-EB11-811F-001DD8B72B62', 'HR Director')]
['Create an email written for 9D75B0F8-72D7-EB11-8121-001DD8B72B61 that executes the following function: Operations & Enzblement Lead but there are no campaigns to recommend.', 'Create an email written for C39094D7-60B1-EB11-811F-001DD8B72B62 that executes the following function: HR Director and recommend the following campaigns: OV-NW-Voka Netwekkers bij Well Played.']


In [26]:
print(get_answer(text))

Subject: Recommended Campaigns for Verantwoordelijke Personeel

Dear Erik Van Vliet,

I hope this email finds you well. I am reaching out to you as the responsible personnel for recommending campaigns that align with your role and objectives. After analyzing your requirements and considering your target audience, I would like to suggest the following campaigns:

1. Campaign ID: D9834556-240C-EE11-8F6E-6045BD8956C9
   This campaign focuses on enhancing employee engagement and fostering a positive work culture. It includes various initiatives such as team-building activities, recognition programs, and wellness initiatives. By implementing this campaign, you can create a more motivated and productive workforce.

2. Campaign ID: B3865100-DEEF-ED11-8849-6045BD895233
   This campaign aims to improve employee training and development. It includes workshops, seminars, and online courses to enhance the skills and knowledge of your staff. By investing in their professional growth, you can ensure